## **Term Weighting Using TF-RF Dataset 1**

In [ ]:
import pandas as pd
import numpy as np


### **Stemmed Dataset**

In [ ]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.head(10)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


- **_mendefinisikan fungsi untuk perhitungan tf dan df_**


In [ ]:
# Fungsi untuk menampilkan seluruh data teks pada kolom
def showAll(column):
    for value in column:
        print(value)


# Fungsi untuk menampilkan data pada kategori tertentu saja
def showOneCategory(typeColumn, textColumn, columnType):
    for text, category in zip(textColumn, typeColumn):
        if category == columnType:
            print(text)


# Fungsi untuk memeriksa apakah suatu kata ada dalam array atau tidak (untuk menghindari duplikasi)
def check(array, word):
    return word not in array


# Fungsi untuk menggabungkan semua data hasil pra-pemrosesan menjadi daftar kata tunggal (tanpa duplikasi)
def termMerging(textColumn, termList):
    for text in textColumn:
        for term in text:
            if check(termList, term):
                termList.append(term)


# Fungsi untuk menghitung frekuensi kemunculan setiap kata dalam setiap dokumen
def countTF(textColumn, typeColumn, termList, TFList):
    for text, category in zip(textColumn, typeColumn):
        docList = [text.count(term) for term in termList]
        docList.append(category)
        TFList.append(docList)


# Fungsi untuk mengubah nilai yang lebih besar dari 1 menjadi 1 (untuk perhitungan DF)
def changeValue(TFList, prepareDFList):
    for doc in TFList:
        dataList = [min(term, 1) for term in doc[:-1]]
        prepareDFList.append(dataList)


# Fungsi untuk menghitung dalam berapa banyak dokumen suatu kata muncul (DF)
def countDF(TFList, DFList):
    for term in zip(*TFList):
        DFList.append(sum(term))


In [ ]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [ ]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


In [ ]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


In [ ]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [ ]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [ ]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


In [ ]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


In [ ]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


In [ ]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


In [ ]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


In [ ]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


In [ ]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


In [ ]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


In [ ]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


In [ ]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)


In [ ]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))


In [ ]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))


### **_Save Dataframe To CSV_**


In [ ]:
columnName = term
columnName.append('sentiment')

In [ ]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF.csv", index=False, sep="\t")


### **_Unstemmed Dataset_**


- _calculate TF, RF & TF-RF for unstemmed dataset to compare it later_


In [ ]:
import pandas as pd

In [ ]:
df_noStem = pd.read_csv(
    "../dataset/INA_TweetsPPKM_Preprocessed_NoStemming.csv", sep="\t"
)


In [ ]:
df_noStem.head()

## **Term Weighting Using TF-RF Dataset 2**

In [ ]:
import pandas as pd
import numpy as np


### **Stemmed Dataset**

In [ ]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS2.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


In [ ]:
df.drop(['date','user'],axis=1,inplace=True)

- **_mendefinisikan fungsi untuk perhitungan tf dan df_**


In [ ]:
# Fungsi untuk menampilkan seluruh data teks pada kolom
def showAll(column):
    for value in column:
        print(value)


# Fungsi untuk menampilkan data pada kategori tertentu saja
def showOneCategory(typeColumn, textColumn, columnType):
    for text, category in zip(textColumn, typeColumn):
        if category == columnType:
            print(text)


# Fungsi untuk memeriksa apakah suatu kata ada dalam array atau tidak (untuk menghindari duplikasi)
def check(array, word):
    return word not in array


# Fungsi untuk menggabungkan semua data hasil pra-pemrosesan menjadi daftar kata tunggal (tanpa duplikasi)
def termMerging(textColumn, termList):
    for text in textColumn:
        for term in text:
            if check(termList, term):
                termList.append(term)


# Fungsi untuk menghitung frekuensi kemunculan setiap kata dalam setiap dokumen
def countTF(textColumn, typeColumn, termList, TFList):
    for text, category in zip(textColumn, typeColumn):
        docList = [text.count(term) for term in termList]
        docList.append(category)
        TFList.append(docList)


# Fungsi untuk mengubah nilai yang lebih besar dari 1 menjadi 1 (untuk perhitungan DF)
def changeValue(TFList, prepareDFList):
    for doc in TFList:
        dataList = [min(term, 1) for term in doc[:-1]]
        prepareDFList.append(dataList)


# Fungsi untuk menghitung dalam berapa banyak dokumen suatu kata muncul (DF)
def countDF(TFList, DFList):
    for term in zip(*TFList):
        DFList.append(sum(term))


In [ ]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [ ]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


In [ ]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


In [ ]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [ ]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [ ]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


In [ ]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


In [ ]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


In [ ]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


In [ ]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


In [ ]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


In [ ]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


In [ ]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


In [ ]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


In [ ]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)


In [ ]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))


In [ ]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))


### **_Save Dataframe To CSV_**


In [ ]:
columnName = term
columnName.append('sentiment')

In [ ]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS2.csv", index=False, sep="\t")


## **Term Weighting Using TF-RF Dataset 3**

In [2]:
import pandas as pd
import numpy as np


### **Stemmed Dataset**

In [3]:
load_path = "../dataset/INA_TweetsPPKM_Preprocessed_DS3.csv"
# Menentukan path file dataset yang akan di-load

df = pd.read_csv(load_path, sep="\t")
# Membaca dataset dari file CSV menggunakan pandas

df["tweet"] = df["tweet"].apply(lambda word: eval(word))
# Menggunakan fungsi `apply` pada kolom "tweet" untuk menerapkan fungsi lambda pada setiap kata
# Fungsi lambda digunakan untuk mengkonversi string kata-kata menjadi list kata-kata dengan menggunakan fungsi `eval`

df.sample(5)
# Menampilkan 10 baris pertama dari dataframe yang sudah diubah


,tweet,sentiment
382,"[hayu, sen, jens, snack, gaiscemilan, enak, ni...",1
1265,"[booming, tahan, pangan, jangan, lupa, apa, te...",1
6652,"[bila, beri, orang, ikhlas, apa, makna, nya]",0
6020,"[ppkm, level, saing, sama, mak, icih, bentar]",0
4953,"[ppkm, panjang, sedih, lihat, dagang, yg, cari...",0


In [5]:
df.drop(['date','user'],axis=1,inplace=True,errors='ignore')

- **_mendefinisikan fungsi untuk perhitungan tf dan df_**


In [7]:
# Fungsi untuk menampilkan seluruh data teks pada kolom
def showAll(column):
    for value in column:
        print(value)


# Fungsi untuk menampilkan data pada kategori tertentu saja
def showOneCategory(typeColumn, textColumn, columnType):
    for text, category in zip(textColumn, typeColumn):
        if category == columnType:
            print(text)


# Fungsi untuk memeriksa apakah suatu kata ada dalam array atau tidak (untuk menghindari duplikasi)
def check(array, word):
    return word not in array


# Fungsi untuk menggabungkan semua data hasil pra-pemrosesan menjadi daftar kata tunggal (tanpa duplikasi)
def termMerging(textColumn, termList):
    for text in textColumn:
        for term in text:
            if check(termList, term):
                termList.append(term)


# Fungsi untuk menghitung frekuensi kemunculan setiap kata dalam setiap dokumen
def countTF(textColumn, typeColumn, termList, TFList):
    for text, category in zip(textColumn, typeColumn):
        docList = [text.count(term) for term in termList]
        docList.append(category)
        TFList.append(docList)


# Fungsi untuk mengubah nilai yang lebih besar dari 1 menjadi 1 (untuk perhitungan DF)
def changeValue(TFList, prepareDFList):
    for doc in TFList:
        dataList = [min(term, 1) for term in doc[:-1]]
        prepareDFList.append(dataList)


# Fungsi untuk menghitung dalam berapa banyak dokumen suatu kata muncul (DF)
def countDF(TFList, DFList):
    for term in zip(*TFList):
        DFList.append(sum(term))


In [8]:
term = []
# Membuat list kosong untuk menyimpan semua term

termMerging(termList=term, textColumn=df["tweet"])
# Memanggil fungsi `termMerging` dengan parameter `termList` yang merupakan list kosong yang akan diisi dengan semua term
# Parameter `textColumn` adalah kolom "tweet" dari dataframe `df`
# Fungsi `termMerging` akan menggabungkan semua term dari kolom "tweet" ke dalam list `term` tanpa adanya term yang berulang


In [9]:
term_frequency = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen

countTF(df["tweet"], df["sentiment"], term, term_frequency)
# Memanggil fungsi `countTF` dengan parameter sebagai berikut:
# - `textColumn`: kolom "tweet" dari dataframe `df` yang berisi teks dalam setiap dokumen
# - `typeColumn`: kolom "sentiment" dari dataframe `df` yang berisi label sentimen untuk setiap dokumen
# - `termList`: list yang berisi semua term yang telah dihasilkan sebelumnya
# - `TFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dan label sentimennya

print(len(term_frequency))
# Menampilkan panjang dari list `term_frequency`, yang mengindikasikan jumlah dokumen yang diproses


7896


In [10]:
tf_forDF = []
# Membuat list kosong untuk menyimpan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

changeValue(term_frequency, tf_forDF)
# Memanggil fungsi `changeValue` dengan parameter sebagai berikut:
# - `TFList`: list `term_frequency` yang berisi frekuensi term dalam setiap dokumen dan label sentimennya
# - `prepareDFList`: list yang akan diisi dengan frekuensi term dalam setiap dokumen dengan nilai yang telah diubah

print(len(tf_forDF))
# Menampilkan panjang dari list `tf_forDF`, yang mengindikasikan jumlah dokumen yang diproses setelah nilai yang lebih besar dari 1 diubah menjadi 1


7896


In [11]:
document_frequency = []
# Membuat list kosong untuk menyimpan frekuensi kemunculan setiap term dalam dokumen

countDF(tf_forDF, document_frequency)
# Memanggil fungsi `countDF` dengan parameter sebagai berikut:
# - `TFList`: list `tf_forDF` yang berisi frekuensi term dalam setiap dokumen dengan nilai yang telah diubah
# - `DFList`: list yang akan diisi dengan frekuensi kemunculan setiap term dalam dokumen

print(len(document_frequency))
# Menampilkan panjang dari list `document_frequency`, yang mengindikasikan jumlah term yang muncul dalam dokumen


15940


### **_Find value for RF & TF-RF_**

<ol>
  <li>RF</li>
  <li>TF-RF</li>

</ol>


In [12]:
# method to count TF in each document category
def count_tf_category(dataTableCategory, termList, textColumn):
    tf_spam_list = []
    for i in range(len(dataTableCategory)):
        doclist = []
        for j in termList:
            count = 0
            for k in range(len(textColumn[i])):
                if j == textColumn[i][k]:
                    count += 1
            doclist.append(count)
        doclist.append(dataTableCategory["sentiment"][i])
        tf_spam_list.append(doclist)
    return tf_spam_list


In [13]:
# splitting tweets that only contain positive sentiment
df_positive = df[df["sentiment"] != 0].reset_index(drop=True)
df_positive.head()


,tweet,sentiment
0,"[hadir, boss, mega, win, jackpot, juta, rupiah...",1
1,"[hayolohhhhhhhhh, neng, kannnn]",1
2,"[thank, you, temanteman, udah, mampir, ngobrol...",1
3,"[ruwet, ruwet, ruwet, ini, negeri, ruwetnesia,...",1
4,"[film, bagus, nih, karya, habaib, tonton, dah]",1


In [14]:
# splitting tweets that only contain negative sentiment
df_negative = df[df["sentiment"] != 1].reset_index(drop=True)
df_negative.head()


,tweet,sentiment
0,"[stimulasi, optimal, laku, online, penuh, butu...",0
1,"[ye, ampun, mbh, orang, wapres, lohampgelar, o...",0
2,"[mulaik, kumat]",0
3,"[welcome, ppkm, bau, bau, naik, level, nih, gi...",0
4,"[sungguh, allah, cipta, hamba, makhluk, lemah,...",0


In [15]:
# count tf value in tweets with positive sentiment
tf_positive_rf = count_tf_category(df_positive, term, df_positive["tweet"])
print(len(tf_positive_rf))


3916


In [16]:
# count tf value in tweets with negative sentiment
tf_negative_rf = count_tf_category(df_negative, term, df_negative["tweet"])
print(len(tf_negative_rf))


3980


In [17]:
tf_positive_df = []
changeValue(tf_positive_rf, tf_positive_df)
print(len(tf_positive_df))


3916


In [18]:
tf_negative_df = []
changeValue(tf_negative_rf, tf_negative_df)
print(len(tf_negative_df))


3980


In [19]:
nij = []
countDF(tf_positive_df, nij)
print(len(nij))


15940


In [20]:
nij_aksen = []
countDF(tf_negative_df, nij_aksen)
print(len(nij_aksen))


15940


In [21]:
# determining the maximum value between 1 and the value of nij accent
max_numerator = []
for i in range(len(nij_aksen)):
    temp = 0
    temp = max(1, nij_aksen[i])
    max_numerator.append(temp)
print(max_numerator)


[4, 3, 1, 1, 1, 17, 4, 13, 2, 3, 5, 1, 6, 1, 18, 1, 1, 7, 1, 189, 1, 1, 18, 7, 6, 52, 6, 10, 1, 1, 13, 34, 1, 3, 64, 83, 3, 2, 34, 1, 1, 1, 53, 512, 1, 196, 6, 9, 117, 3, 1, 10, 54, 1, 4, 24, 5, 126, 94, 48, 59, 21, 162, 434, 36, 1, 1, 1, 1, 63, 2, 3, 5, 127, 7, 1, 25, 1, 1, 83, 1, 33, 1067, 3, 14, 13, 1, 16, 102, 44, 1, 29, 2, 4, 38, 46, 42, 52, 17, 91, 9, 13, 33, 1, 5, 1, 15, 57, 184, 1, 8, 3, 37, 1, 216, 5, 2, 1, 5, 17, 49, 8, 10, 91, 2, 8, 1, 55, 212, 4, 21, 70, 9, 227, 292, 76, 19, 7, 24, 89, 10, 5, 1, 14, 117, 86, 57, 4, 12, 5, 7, 195, 17, 1, 1, 14, 38, 13, 40, 167, 41, 8, 14, 138, 21, 3, 45, 8, 18, 145, 1, 31, 152, 16, 7, 1, 26, 13, 4, 24, 22, 1, 28, 5, 8, 11, 46, 33, 6, 1, 42, 10, 141, 1, 17, 43, 1, 8, 49, 35, 1, 19, 42, 114, 29, 56, 4, 51, 1, 1, 1, 6, 33, 1, 31, 1, 12, 6, 16, 271, 28, 2, 11, 46, 26, 3, 2, 33, 15, 1, 1, 1, 1, 49, 47, 4, 321, 393, 122, 96, 5, 20, 1, 80, 83, 43, 28, 228, 6, 199, 1, 16, 20, 8, 1, 30, 8, 128, 18, 1, 10, 3, 2, 2, 2, 1, 1, 9, 1, 109, 2, 20, 2, 89, 10

In [22]:
import math

# find rf value
rf_list = []
for i in range(len(nij)):
    temp = 0
    temp = math.log10(2 + (nij[i] / max_numerator[i]))
    temp = round(temp, 5)
    rf_list.append(temp)
print(rf_list)


[1.09691, 0.77815, 1.14613, 1.27875, 1.14613, 0.5404, 0.8893, 0.74938, 1.21748, 0.97004, 1.23553, 1.14613, 1.06695, 0.60206, 0.53712, 0.60206, 0.8451, 0.7231, 1.14613, 0.41727, 1.34242, 0.47712, 0.77411, 1.17609, 1.19498, 0.69224, 0.47712, 0.716, 1.20412, 0.77815, 0.45426, 0.52543, 0.60206, 1.09108, 0.39522, 0.43117, 0.47712, 0.60206, 0.50595, 0.60206, 0.60206, 0.60206, 0.5237, 0.38275, 0.60206, 0.40147, 0.61979, 0.90908, 0.65527, 1.12494, 0.60206, 0.5563, 0.40115, 0.60206, 0.94201, 0.58358, 0.5563, 0.59336, 0.45175, 0.64098, 0.36269, 0.54701, 0.46991, 0.46115, 0.63682, 0.69897, 0.77815, 0.47712, 0.60206, 0.59686, 0.47712, 0.66901, 0.53148, 0.51004, 0.45593, 0.69897, 0.44716, 0.60206, 0.77815, 0.63237, 0.60206, 0.68831, 0.62302, 0.727, 0.6173, 0.55815, 0.47712, 0.8451, 0.39623, 0.96185, 1.47712, 0.50139, 1.14613, 0.97772, 0.59631, 0.86358, 0.52597, 0.79185, 0.57573, 0.46256, 0.60206, 0.71213, 0.45461, 0.77815, 1.11394, 1.43136, 0.83251, 0.52516, 0.5809, 1.50515, 0.54407, 0.90309, 0.481

In [23]:
# Combining the values of TF for spam and non-spam.
tf_forRF = []

for i in range(len(tf_positive_rf)):
    tf_forRF.append(tf_positive_rf[i])

for i in range(len(tf_negative_rf)):
    tf_forRF.append(tf_negative_rf[i])

print(len(tf_forRF))


7896


In [24]:
tf_rf = []
for i in range(len(tf_forRF)):
    doclist = []
    for j in range(len(tf_forRF[0])):
        if j == len(tf_forRF[0]) - 1:
            temp = tf_forRF[i][j]
            doclist.append(temp)
        else:
            temp = rf_list[j] * tf_forRF[i][j]
            doclist.append(temp)
    tf_rf.append(doclist)
print(len(tf_rf))


7896


### **_Save Dataframe To CSV_**


In [25]:
columnName = term
columnName.append('sentiment')

In [26]:
data = pd.DataFrame(tf_rf, columns=columnName)
data.to_csv("../dataset/INA_TweetsPPKM_TFRF_DS3.csv", index=False, sep="\t")
